<a href="https://colab.research.google.com/github/PratiteeMalakar/hello-world/blob/main/Centralorchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# Generate sample data for demand forecasting
dates = pd.date_range(start='2023-01-01', periods=365, freq='D')
products = ['Product_A', 'Product_B', 'Product_C']

data = []
for product in products:
    sales = np.random.poisson(lam=20, size=len(dates))
    data.extend(zip([product]*len(dates), dates, sales))

df_sales = pd.DataFrame(data, columns=['Product', 'Date', 'Sales'])
df_sales.to_csv('sales_data.csv', index=False)

# Generate sample data for inventory levels
inventory_data = {'Product': products, 'Inventory_Level': [100, 150, 200]}
df_inventory = pd.DataFrame(inventory_data)
df_inventory.to_csv('inventory_data.csv', index=False)

# Generate sample data for logistics
logistics_data = {
    'Product': products,
    'Warehouse': ['W1', 'W2', 'W3'],
    'Transportation_Cost': [10, 15, 20]
}
df_logistics = pd.DataFrame(logistics_data)
df_logistics.to_csv('logistics_data.csv', index=False)


In [3]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load data
df_sales = pd.read_csv('sales_data.csv')

# Feature engineering
df_sales['DayOfYear'] = pd.to_datetime(df_sales['Date']).dt.dayofyear
X = df_sales[['DayOfYear']]
y = df_sales['Sales']

# Train model
model = LinearRegression()
model.fit(X, y)

# Save the model
import joblib
joblib.dump(model, 'demand_forecasting_model.pkl')


['demand_forecasting_model.pkl']

In [4]:
def optimize_inventory(demand_forecast, current_inventory):
    reorder_point = demand_forecast.mean() * 1.5
    optimal_inventory = reorder_point - current_inventory
    return max(0, optimal_inventory)

# Example usage
current_inventory = 100
demand_forecast = np.array([20, 22, 25, 18, 30])
optimal_inventory = optimize_inventory(demand_forecast, current_inventory)
print(f'Optimal inventory to order: {optimal_inventory}')


Optimal inventory to order: 0


In [5]:
def optimize_logistics(inventory_plan, logistics_data):
    logistics_plan = {}
    for product, qty in inventory_plan.items():
        warehouse = logistics_data[logistics_data['Product'] == product]['Warehouse'].values[0]
        logistics_plan[product] = {'Warehouse': warehouse, 'Quantity': qty}
    return logistics_plan

# Example usage
inventory_plan = {'Product_A': 50, 'Product_B': 70}
logistics_data = pd.read_csv('logistics_data.csv')
logistics_plan = optimize_logistics(inventory_plan, logistics_data)
print(logistics_plan)


{'Product_A': {'Warehouse': 'W1', 'Quantity': 50}, 'Product_B': {'Warehouse': 'W2', 'Quantity': 70}}


In [6]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np

app = Flask(__name__)

# Load models
demand_model = joblib.load('demand_forecasting_model.pkl')

@app.route('/forecast', methods=['GET'])
def forecast():
    product = request.args.get('product')
    days_ahead = int(request.args.get('days_ahead'))

    # Load sales data
    df_sales = pd.read_csv('sales_data.csv')
    df_product = df_sales[df_sales['Product'] == product]
    latest_day = pd.to_datetime(df_product['Date']).dt.dayofyear.max()

    # Forecast demand
    future_days = np.array(range(latest_day + 1, latest_day + 1 + days_ahead)).reshape(-1, 1)
    demand_forecast = demand_model.predict(future_days)

    return jsonify({'product': product, 'forecast': demand_forecast.tolist()})

@app.route('/optimize_inventory', methods=['POST'])
def optimize_inventory_route():
    data = request.get_json()
    product = data['product']
    current_inventory = data['current_inventory']

    # Load sales data
    df_sales = pd.read_csv('sales_data.csv')
    df_product = df_sales[df_sales['Product'] == product]

    # Forecast demand
    days_ahead = 30
    latest_day = pd.to_datetime(df_product['Date']).dt.dayofyear.max()
    future_days = np.array(range(latest_day + 1, latest_day + 1 + days_ahead)).reshape(-1, 1)
    demand_forecast = demand_model.predict(future_days)

    # Optimize inventory
    optimal_inventory = optimize_inventory(demand_forecast, current_inventory)

    return jsonify({'product': product, 'optimal_inventory': optimal_inventory})

@app.route('/optimize_logistics', methods=['POST'])
def optimize_logistics_route():
    data = request.get_json()
    inventory_plan = data['inventory_plan']

    # Load logistics data
    logistics_data = pd.read_csv('logistics_data.csv')

    # Optimize logistics
    logistics_plan = optimize_logistics(inventory_plan, logistics_data)

    return jsonify(logistics_plan)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [7]:
# Forecast demand for Product_A for the next 30 days
curl "http://127.0.0.1:5000/forecast?product=Product_A&days_ahead=30"

# Optimize inventory for Product_A with current inventory level of 100
curl -X POST -H "Content-Type: application/json" -d '{"product": "Product_A", "current_inventory": 100}' "http://127.0.0.1:5000/optimize_inventory"

# Optimize logistics for an inventory plan
curl -X POST -H "Content-Type: application/json" -d '{"inventory_plan": {"Product_A": 50, "Product_B": 70}}' "http://127.0.0.1:5000/optimize_logistics"


SyntaxError: invalid syntax (<ipython-input-7-8edee00c6cc3>, line 2)